In [1]:
import os

In [2]:
%pwd

'e:\\Projects for portfolio\\Text Summarization\\research'

In [3]:
#Since I want to get into the root directory i.e Text Summarization using Hugging Face API
os.chdir("../")

In [4]:
%pwd

'e:\\Projects for portfolio\\Text Summarization'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig: # defined for the config components present in artifacts for data transformation
    root_dir : Path 
    data_path : Path
    tokenizer_name : Path

In [7]:
# Configuration manager
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
    # Here we are reading the yaml file and we can now use the file paths present inside pararms and config.yaml        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) # Here we are calling the artifacts_root key values using '.' , which was the purpose of @ensure_annotations

    def get_data_transformation_config(self) -> DataTransformationConfig: # Here we are using the entity to specify the return type classes to make sure proper output is returned
        config= self.config.data_transformation # Calling the data_validation dictionary created in config.yaml file

        create_directories([config.root_dir]) # Creating a directory using the root directory

        data_transformation_config = DataTransformationConfig( # Extracting the values from the config.yaml to here inside data_ingestion_config
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [9]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

[2024-01-26 20:07:30,020: INFO: config: PyTorch version 2.1.2 available.]


In [11]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig): # It will take the configuration from DataIngestionConfig defined earlier , which will in turn use Configuration Manager to take data from config.yaml
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def convert_examples_to_features(self,example_batch): # Used for tokenization of input and target

        #Tokenizing the 'dialogue' sequences of the dataset
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        #Tokenizing the 'summary' sequences of the dataset using the target tokenizer
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        #Returning the input and target encodings done
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True) # Tokenizing the dataset
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset_tokenized"))

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config() # Storing the configuration
    data_transformation = DataTransformation(config=data_transformation_config) # Using the configuration saved earlier to call model_transformation
    data_transformation.convert()
except Exception as e:
    raise e

[2024-01-26 20:14:55,623: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-26 20:14:55,623: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-26 20:14:55,632: INFO: common: created directory at: artifacts]
[2024-01-26 20:14:55,632: INFO: common: created directory at: artifacts/data_transformation]


tokenizer_config.json: 100%|██████████| 88.0/88.0 [00:00<00:00, 60.6kB/s]
c:\Users\arany\anaconda3\envs\textS\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arany\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config.json: 100%|████████